# Iterative IK example

Traditional approach to the inverse kinematics resolution using the Jacobina $J$ approach. 

Ginen a target orientation $R_t$, the current platform orientation $R_c$ and the corresponding motor positions $\theta$.

We can calculate the orientation error vector $e = R_cR_t^T$ (if SO3 matrices). Then we can define the velocity $\dot{e}$ in the orientation space that brings us from the current $R_c$ to the target orientation $R_t$ 

$$
\dot{e} = \frac{e}{\Delta t}
$$

And then we can calculate the motor velocity $\dot{\theta}$ that corresponds to the $\dot{e}$ that brings us to the target $R_t$ using the Jacobian.

$$
\dot{\theta} = J^{-1}(\theta) \dot{e}
$$

As Jacobian ($J(\theta)$) is highly dependant on the motor positions $\theta$ it is common to use a relatively small veclocity $\dot{e}$ (large $\Delta t$) in order to make the movement smooth. We can also write the equation in this way:

$$
\dot{\theta} = \alpha J^{-1}(\theta)e
$$

Where $\alpha$ is typically is small, and correpsonds to $\alpha = 1/\Delta t$

In [1]:
import numpy as np
import time
from poulpe_ethercat_py import PyEthercatServer
from orbita3d import Orbita3dController
from scipy.spatial.transform import Rotation as R

In [2]:
# Start the EtherCAT master
server = PyEthercatServer()
server.launch_server("../../orbita3d_controller/config/ethercat.yaml")
# give some time for the master to start
time.sleep(1)

POULPE controller ready!


In [3]:
# connect to the actuator
orbita = Orbita3dController.from_config("../../orbita3d_controller/config/ethercat_poulpe.yaml")

In [4]:
# orbita kinematic model
from orbita3d import KinematicsModel
kin = KinematicsModel(
    alpha=np.deg2rad(54.0),
    gamma_min= 0.6981317007977318, # 40 degrees
    offset=0.0,
    beta=np.deg2rad(90.0),
    gamma_max=np.deg2rad(175.0),
    passiv_arms_direct=True,
)

In [5]:
# disable the actuator
orbita.disable_torque()
orbita.set_control_mode(3)
# enable the actuator
orbita.enable_torque(reset_target=True)
time.sleep(1)

## The example IK code

In [18]:
# target orientation
target_rpy = np.random.uniform([-30]*3,[30]*3)

# scaling factor 
alpha = 3.0

orbita.enable_torque(reset_target=True)
t0 = time.time()
while time.time()- t0 < 3:
    # current orientation
    current_rpy = R.from_quat(orbita.get_current_orientation()).as_euler('xyz',degrees=True)
    # orientation error
    error = target_rpy - current_rpy
    # velocity to go from the current to the taget orentation in 1s (angle-axis representation)
    vel = R.from_euler('xyz', error, degrees=True).as_rotvec()
    # set the velocity to the motors  motor_vel = J_inv*vel
    orbita.set_target_velocity(vel*alpha)
    time.sleep(0.005)
orbita.set_target_velocity([0]*3)
orbita.disable_torque()

print("target: ", target_rpy, "degrees")
print("reached: ",
R.from_quat(orbita.get_current_orientation()).as_euler('xyz', degrees=True), "degrees")

target:  [-7.25710743  2.41503319  8.0300826 ] degrees
reached:  [-7.21875454  2.37148476  7.9484029 ] degrees
